In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import torch
from sklearn.datasets import make_regression
from torch.distributions.multivariate_normal import MultivariateNormal
seed = 7
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
n_data = 2000
sig1 = torch.ones(8)
sig2 = torch.ones(40,dtype=torch.float) * pow(2,-10)
sig = torch.diag(torch.cat((sig1,sig2)))
x_dist = MultivariateNormal(torch.zeros(48), sig)
x = [x_dist.sample() for i in range(n_data)]
x = torch.stack(x)
w = torch.rand(48,dtype=torch.float)
y = x@w

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
x_train = x.clone().detach().requires_grad_(True)
y_train = y.clone().detach().requires_grad_(True)
train_ds = TensorDataset(x_train, y_train)

In [ ]:
import torch.nn.functional as F
import torch.nn as nn
import mass
device = "cuda:0"

In [ ]:
def fit(model_instance, loss_fn, optim, data_loader, n_iter = 10, epsilon = 0.1):
    iter_count = 0
    epo_count = 0
    while True:
        model_instance.train()
        running_loss = 0.0

        for xb,yb in data_loader:
            iter_count += 1

            xb = xb.to(device)
            yb = yb.to(device)

            pred = model_instance(xb)
            pred = pred.squeeze(1)

            loss = loss_fn(pred, yb)
            loss.backward()
            optim.step()
            optim.zero_grad()
            running_loss += loss.item()

        epo_count += 1
        if running_loss < epsilon:
            return iter_count

In [ ]:
model = nn.Linear(48, 1).to(device)
batch_size = 1
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
opt = mass.Mass(model.parameters(),lr=0.1,alpha=0.02,kappa_t=6)
loss_function = F.mse_loss

In [ ]:
iters = fit(model, loss_function,opt,train_dl,n_iter=40, epsilon = 0.1)

In [ ]:
iter_counts = []
for m in range(10,160,10):
    print("batch size: %d" % (m))
    model = nn.Linear(48, 1).to(device)
    opt = mass.Mass(model.parameters(),lr=0.1,alpha=0.02,kappa_t=6)
    train_dl = DataLoader(train_ds, m, shuffle=True)
    iter_ = fit(model, loss_function,opt,train_dl,n_iter=40, epsilon = 0.1)
    iter_counts.append(iter_)

In [ ]:
s_m = [1/i for i in iter_counts]

In [ ]:
plt.plot(s_m)